In [0]:
strt1="id string, firstname string, lastname string, age string, profession string"
rawdf1=spark.read.schema(strt1).csv(path=["/Volumes/workspace/wd36schema/ingestion_volume/source/"],recursiveFileLookup=True,pathGlobFilter="custsmodified_N*")
strt2="id string, firstname string, age string, profession string,city string"
rawdf2=spark.read.schema(strt2).csv(path=["/Volumes/workspace/wd36schema/ingestion_volume/source/"],recursiveFileLookup=True,pathGlobFilter="custsmodified_T*")
display(rawdf1)
display(rawdf2)
rawdf_merged=rawdf1.unionByName(rawdf2)#Use union only if the dataframes are having same columns in the same order with same datatype..
display(rawdf_merged)

In [0]:
struct1="id string, firstname string, lastname string, age string, profession string"

rawdf1=spark.read.schema(struct1).csv(path=["/Volumes/workspace/wd36schema/ingestion_volume/source/sple_10000_records.csv","/Volumes/workspace/wd36schema/ingestion_volume/source/custsmodified"])

display(rawdf1)

In [0]:
strct1= "productId string, productName  string, stdCost double, stdPrice double, effDt timestamp, dt date"
df1=spark.read.schema(strct1).csv(path="/Volumes/workspace/wd36schema/ingestion_volume/sales.csv",sep=",",header=True, multiLine=True, escape="~", comment='#', ignoreLeadingWhiteSpace=True, ignoreTrailingWhiteSpace=True)
print(df1.schema)

display(df1)

In [0]:
df_cust = spark.read.csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer.csv",header=True,inferSchema=True,sep=",")
display(df_cust)

df_cust.write.format("delta").save("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer_out_delta/", mode="overwrite")

In [0]:
%sql
CREATE TABLE telecom_catalog_assign.landing_zone.customer_managed1 (
    customer_id INT,
    name STRING,
    age INT,
    city STRING,
    plan STRING
)
USING DELTA;

SELECT * FROM telecom_catalog_assign.landing_zone.customer_managed1


In [0]:
df_cust = spark.read.csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer.csv", header=True, inferSchema=True).toDF("customer_id", "name", "age", "city", "plan")
df_cust.write.saveAsTable("telecom_catalog_assign.landing_zone.customer_managed2")
df_cust.write.insertInto(
    "telecom_catalog_assign.landing_zone.customer_managed2",
    overwrite=True
)

display(spark.sql("SELECT * FROM telecom_catalog_assign.landing_zone.customer_managed2"))


In [0]:
%sql
DESCRIBE telecom_catalog_assign.landing_zone.customer_managed;


In [0]:
#1: Write customer data using insertInto() in a new table and find the behavior
#table created for customer
read_customer_df = spark.read.csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer.csv", sep=",").toDF("id","name","age","city","sim_type") 
read_customer_df.write.saveAsTable("telecom_catalog_assign.landing_zone.custtbl",mode='overwrite')
#insert into customer table already created
dlt_tb = spark.read.csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/custs.csv").toDF("id", "name", "age", "city", "sim_type")
dlt_tb.write.insertInto("telecom_catalog_assign.landing_zone.custtbl")
display(spark.sql("select * from telecom_catalog_assign.landing_zone.custtbl"))

#2:Write usage data using insertTable() with overwrite mode
read_usage = spark.read.options(header="True",sep="\t").csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/usage/usage_new.csv") 
read_usage.write.insertInto("telecom_catalog_assign.landing_zone.usagetbl",overwrite=True)
spark.sql("select * from telecom_catalog_assign.landing_zone.usagetbl").show()

In [0]:
from pyspark.sql import SparkSession

# Create Spark session
spark = SparkSession.builder \
    .appName("DeltaOverwriteExample") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()

# Sample DataFrame
data = [
    (108, "John",52,"Mumbai","POSTPAID"),
    (109,"Jane",20,"Delhi","PREPAID"),
    (110,"Alice",25,"Pune","POSTPAID"),
    (111,"Bob",30,"Bangalore","PREPAID"),

]

columns = ["id", "name", "age", "city", "plan"]

df = spark.createDataFrame(data, columns)

# Path to Delta table
delta_path = "/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer_out_delta/"

# Write DataFrame in Delta format using overwrite mode
df.write.format("delta").option("mergeSchema", "true") \
    .option("multiline", "true") \
    .mode("overwrite") \
    .save(delta_path)

spark.read.format("delta").load(delta_path).count()

print("Data written to Delta table with overwrite mode successfully.")


In [0]:
data = [
    (101, "John",52,"Mumbai","POSTPAID"),
    (103,"Jane",20,"Delhi","PREPAID"),
    (102,"Alice",25,"Pune","POSTPAID"),
    (107,"Bob",30,"Bangalore","PREPAID"),
    (106,"Bob",30,"Bangalore","PREPAID")
]

columns = ["id", "name", "age", "city", "plan"]

df = spark.createDataFrame(data, columns)

df.write.format("delta").mode("append").save("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer_practice/")
df1 = spark.read.format("delta").load("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer_practice/")
display(df1)
df1.show()

In [0]:
from pyspark.sql import SparkSession

# Create Spark session
spark = SparkSession.builder \
    .appName("DeltaOverwriteExample") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .getOrCreate()

# Sample DataFrame
data = [
    (108, "John",52,"Mumbai","POSTPAID"),
    (109,"Jane",20,"Delhi","PREPAID"),
    (110,"Alice",25,"Pune","POSTPAID"),
    (111,"Bob",30,"Bangalore","PREPAID"),
]

columns = ["id", "name", "amt"]

df = spark.createDataFrame(data, columns)

# Path to Delta table
delta_path = "/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer.csv"

# Write DataFrame in Delta format using overwrite mode
df.write.format("csv").mode("overwrite").save(delta_path)

# print("Data written to Delta table with overwrite mode successfully.")
# df.write.format("delta").option("mergeSchema", "true") \
#     .mode("overwrite") \
#     .save(delta_path)

print("Data written to Delta table with overwrite mode successfully.")


In [0]:
df1 = spark.read.format("delta").load("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer_out_delta/X.c000.snappy.parquet")
# df1.show()
df1.write.format("delta").mode("overwrite").save("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer_out_delta/Y.c000.snappy.parquet")
df1.show()

In [0]:
df1 = spark.read.format("delta").load(
    "/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer_out_delta"
)
df1.write.format("delta").mode('overwrite').option().save(
    "/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer_out_delta_Y"
)
df1.write.delta("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/customer/customer_out_delta_Y", overwriteSchema=True, mode="overwrite")
display(df1)

In [0]:
struct1="custid int,name string,age double,dop date"

df1=spark.read.schema(struct1).csv("/Volumes/workspace/wd36schema/ingestion_volume/malformeddata1.txt",header=False,sep=",",mode='permissive',comment='#',quote="'", escape="|", dateFormat="yyyy-dd-MM", ignoreLeadingWhiteSpace=True, ignoreTrailingWhiteSpace=False, multiLine=True)

cleaned = df1.selectExpr(
    "regexp_replace(name, '\\\\', '~') as name"
)
display(cleaned)
# if we use multiline = true 
# display() - it shows as expanded text area
# show() - it shows as single line with \n
cleaned.show(10,False)

In [0]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

raw = spark.read.text("/Volumes/workspace/wd36schema/ingestion_volume/malformeddata1.txt")

cleaned = raw.selectExpr(
    "regexp_replace(value, '\\\\', '|') as value"
)

cleaned.write.mode("overwrite").text("/tmp/cleaned_csv")
df1 = spark.read.schema(struct1).csv(
    "/tmp/cleaned_csv",
    header=False,
    sep=",",
    escape="|",
    quote="'",
    mode="permissive",
    multiLine=True
)

In [0]:
#When to go for quote: If the data is having delimiter in it..
#When to go for escape: If the data is having quote in it...

struct1="custid int,name string,age double,dop date,corrupt_record string"

df1=spark.read.schema(struct1).csv("/Volumes/workspace/wd36schema/ingestion_volume/malformeddata1.txt",header=False,sep=",",mode='permissive',comment='#',columnNameOfCorruptRecord="corrupt_record",quote="'", escape="|", dateFormat="yyyy-dd-MM", ignoreLeadingWhiteSpace=True, ignoreTrailingWhiteSpace=False)

display(df1)

In [0]:
raw_bytes = b'caf\xc3\xa9'
text = raw_bytes.decode("utf-8")
print(text)


In [0]:
struct1="custid int,name string,age double,dop date"

df1=spark.read.schema(struct1).csv("/Volumes/workspace/wd36schema/ingestion_volume/malformeddata1.txt",header=False,sep=",",mode='permissive',comment='#',quote="'", escape="|", dateFormat="yyyy-dd-MM", ignoreLeadingWhiteSpace=True, ignoreTrailingWhiteSpace=False, multiLine=True)

cleaned = df1.selectExpr(
    "regexp_replace(value, '\\\\', '~') as value"
)
display(df1)
# if we use multiline = true 
# display() - it shows as expanded text area
# show() - it shows as single line with \n
df1.show(10,False)

In [0]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

raw = spark.read.text("/Volumes/workspace/wd36schema/ingestion_volume/malformeddata1.txt")

cleaned = raw.selectExpr(
    "regexp_replace(value, '\\\\', '|') as value"
)

cleaned.write.mode("overwrite").text("/tmp/cleaned_csv")
df1 = spark.read.schema(struct1).csv(
    "/tmp/cleaned_csv",
    header=False,
    sep=",",
    escape="|",
    quote="'",
    mode="permissive",
    multiLine=True
)

In [0]:
struct1="id string, firstname string, lastname string, age string, profession string"
rawdf1=spark.read.schema(struct1).csv(path="/Volumes/workspace/wd36schema/ingestion_volume/source/custsmodified")
cleanseddf=rawdf1.na.drop(how="any")
display(cleanseddf)
print("Total rows without firstname and lastname with null values",len(cleanseddf.collect()))

cleanseddf1=rawdf1.na.drop(how="all")
display(cleanseddf1)
print("Total rows without firstname and lastname with null values",len(cleanseddf1.collect()))

print("any one row in the raw df with age null")
display(rawdf1.where("age is null"))
print("any one row in the cleansed df with age null")
display(cleanseddf.where("age is null"))
cleanseddf=rawdf1.na.drop(how="any",subset=["id","age"])
print("any one row in the cleansed df with id or age null")
display(cleanseddf)
cleanseddf=rawdf1.na.drop(how="all",subset=["firstname","lastname"])
print("any one row in the cleansed df with firstname and lastname is null")
print("Total rows without firstname and lastname with null values",len(cleanseddf.collect()))
display(cleanseddf)

In [0]:
cleanseddf=rawdf1.na.drop(how="any")#how="any" → drop the row if any column has a null value

print("any one row in the raw df with age null")
display(rawdf1.where("age is null"))

print("any one row in the cleansed df with age null")
display(cleanseddf.where("age is null"))#any one column contains null will be cleaned

cleanseddf=rawdf1.na.drop(how="any",subset=["id","age"])#If we need CDE without nulls (Critical Data Elements/Significant columns) columns
print("any one row in the cleansed df with id or age null")
display(cleanseddf)
print("Total rows without id","age with null values",len(cleanseddf.collect()))

cleanseddf=rawdf1.na.drop(how="all",subset=["firstname","lastname"])#4000004,Gretchen,,66, 
print("any one row in the cleansed df with firstname and lastname is null")
print("Total rows without firstname and lastname with null values",len(cleanseddf.collect()))

In [0]:
display(cleanseddf.where("age is null or id is null"))#any
display(cleanseddf.where("age is null and id is null"))#and

In [0]:
data1 = [
    {"id": 1, "firstname": "John", "lastname": "Doe"},
    {"id": 2, "firstname": "Jane", "lastname": "Smith"}
]

data2 = [
    {"id": 3, "firstname": "Mike", "lastname": "Brown", "age": 30},
    {"id": 4, "firstname": "Sara", "lastname": "Wilson", "age": 25},
    {"id": 5, "firstname": "william", "lastname": "Wilson", "age": 35}
]

df1 = spark.createDataFrame(data1)
df2 = spark.createDataFrame(data2)

# Write JSON files
df1.write.mode("overwrite").json("/Volumes/sample_notebook/practice_workspace/volume1/file1")
df2.write.mode("overwrite").json("/Volumes/sample_notebook/practice_workspace/volume1/file2")

json_df = spark.read.json("/Volumes/sample_notebook/practice_workspace/volume1/file*")
json_df.show()

json_df.write.format("delta").mode("append").option("mergeSchema", "true").saveAsTable("customer_delta")

In [0]:
display(scrubbeddf3.where("id in ('4000001')"))#before row level dedup
dedupdf1=scrubbeddf3.distinct()#It will remove the row level duplicates
display(dedupdf1.where("id in ('4000001')"))

print("non prioritized deduplication, just remove the duplicates retaining only the first row")
display(dedupdf1.coalesce(1).where("id in ('4000003')"))#before col level dedup
dedupdf2=dedupdf1.coalesce(1).dropDuplicates(subset=["id"])#It will remove the column level duplicates (retaining the first row in the dataframe)
display(dedupdf2.where("id in ('4000003')"))

In [0]:
from pyspark.sql.functions import lit,initcap,col
#withColumn("stringcolumnname to add in the df",lit('hardcoded')/initcap(col("colname")))
standarddf1=dedupdf2.withColumn("sourcesystem",lit("Retail"))#SparkSQL - DSL(FBP)
display(standarddf1.limit(20))